In [ ]:
from astropy.io import fits
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ckdtree as kdtree 
import itertools
%matplotlib inline

In [ ]:
#read in TGAS data for and matched sample for magnitudes 
tgas = fits.getdata("stacked_tgas.fits", 1)
tgasRave = fits.getdata('tgas-rave.fits', 1)
matchTgasApass = fits.getdata('tgas-matched-apass-dr9.fits')

In [ ]:
#print tgasRave.dtype.names
print tgas.dtype.names
print matchTgasApass.dtype.names

In [ ]:
raveInd = np.in1d(tgas['source_id'], tgasRave['source_id'])

In [ ]:
#print np.array(tgas.dtype.names).T
#print np.array(tgasRave.dtype.names).T

In [ ]:
#read in Adrian's distances from sampling the posterior 
ndist = 100
nfiles = 16
dist = None #np.zeros(len(tgasMatched), ndist)
for j in range(1,nfiles+1):
    with h5py.File("distance-samples-{:02d}.hdf5".format(j)) as f:
        if dist is None:
            dist = f['distance'][:,:ndist]
        else:
            dist = np.concatenate((dist, f['distance'][:,:ndist]))

In [ ]:
#calculate the median distance from the posterior 
medianDist = np.median(dist, axis=1)

In [ ]:
#various cuts to select sample 
proj = 'twins'
ndist = ndist
nNeighbors = 100
sigMax = 0.05
maxDist = 8000. #1000. #pc
fracErrorDistance = 1.

if proj ==  'twins':
    minDist = 0.0 #pc
    parallaxSN = 16.
    galacticLatMin = 30. #degrees
    galacticLatMax = None #degrees
if proj == 'dust':
    minDist = 250. #pc
    parallaxSN = 5.
    galacticLatMin = 0.  #degrees 
    galacticLatMax = 10. #degrees
    
mags = ['bmag', 'gmag', 'vmag', 'rmag', 'imag']
errors = ['e_bmag', 'e_gmag', 'e_vmag', 'e_rmag', 'e_imag']
wavelength = [420., 475., 520., 658., 806.]

In [ ]:
#visualize historgrams of parallax errors 
import matplotlib as mpl
mpl.rcParams.update({'font.size': 22})
fig, axes = plt.subplots(1, 3, figsize=(20,5))
notNan = ~np.isnan(tgas['parallax']) & (tgas['parallax'] > 0.0)
nbins=100
err = np.sqrt(tgas['parallax_error']**2. + 0.3**2.)
out = axes[0].hist(tgas['parallax_error'][notNan], log=True, bins=nbins)
out = axes[1].hist(np.log10(tgas['parallax'][notNan]), log=True, bins=nbins)
out = axes[2].hist(tgas['parallax'][notNan]/err[notNan], log=True, bins=nbins)

axes[0].set_xlabel('Parallax Error')
axes[1].set_xlabel('log Parallax')
axes[2].set_xlabel('Parallax S/N')

In [ ]:
#current cut: no magnitudes are NaNs, 0 < errors < sigMax, 
#print np.sum(tgas['parallax']/err > parallaxSN)

noNans = ~np.isnan(matchTgasApass['bmag']) & ~np.isnan(matchTgasApass['vmag']) & \
         ~np.isnan(matchTgasApass['gmag']) & ~np.isnan(matchTgasApass['rmag']) & \
         ~np.isnan(matchTgasApass['imag'])

posErrors = (matchTgasApass['e_bmag'] > 0) & (matchTgasApass['e_vmag'] > 0) & \
            (matchTgasApass['e_gmag'] > 0) & (matchTgasApass['e_rmag'] > 0) & \
            (matchTgasApass['e_imag'] > 0)

lowPhotError = (matchTgasApass['e_bmag'] < sigMax) & (matchTgasApass['e_vmag'] < sigMax) & \
               (matchTgasApass['e_gmag'] < sigMax) & (matchTgasApass['e_rmag'] < sigMax) & \
               (matchTgasApass['e_imag'] < sigMax)

noDust = (medianDist < maxDist) & (np.abs(tgas['b']) > galacticLatMin)

apassMatch = matchTgasApass['matched']

goodDistance = tgas['parallax']/err > parallaxSN

hasDust = medianDist >= minDist

raveInd = raveInd

if galacticLatMax: inDisk = np.abs(tgas['b'] <= galacticLatMax)

matched = goodDistance & noNans & posErrors & lowPhotError & noDust & apassMatch & raveInd

In [ ]:
tgasMatched = tgas[matched]
magsMatched = matchTgasApass[matched]
errMatched = err[matched]

In [ ]:
print 'Number of tgas stars: ', len(tgas)
print 'Number of matched stars: ', np.sum(matched)
print 'Percent matched = ', 100 - (len(tgas) - np.sum(matched))/np.float(len(tgas))*100., '%'
#print np.min(magsMatched['gmag']), np.max(magsMatched['gmag'])

In [ ]:
distMatched = dist[matched]
distUnmatched = dist[~matched]
medianDistMatched = np.median(distMatched, axis=1)
meanDistMatched = np.mean(distMatched, axis=1)
medianDistUnmatched = np.median(distUnmatched, axis=1)
fig, axes = plt.subplots(1,2, figsize=(15, 5))
out = axes[0].hist(medianDistMatched/1000., bins=50, log=True)
out = axes[1].hist(tgas['parallax'][matched]/err[matched], bins=50, log=True)
#out = axes[1].hist(np.log10(tgasMatched['parallax_error']))
#out = axes[1].hist(magsMatched['gmag'], bins=50, log=True)
axes[0].set_xlabel('Distance [kpc]')
axes[1].set_xlabel('S/N')
plt.tight_layout()
fig.savefig('distanceTgas.png')

In [ ]:
def absMag(apparentMag, distance):
    return apparentMag - 5.*np.log10(distance/10.)

In [ ]:
M_b = absMag(magsMatched['bmag'], medianDistMatched)
M_g = absMag(magsMatched['gmag'], medianDistMatched)
M_v = absMag(magsMatched['vmag'], medianDistMatched)
M_r = absMag(magsMatched['rmag'], medianDistMatched)
M_i = absMag(magsMatched['imag'], medianDistMatched)

In [ ]:
B_V = magsMatched['bmag'] - magsMatched['vmag']
g_r = magsMatched['gmag'] - magsMatched['rmag']
r_i = magsMatched['rmag'] - magsMatched['imag']

In [ ]:
from matplotlib.colors import LogNorm
fig, axes = plt.subplots(2, 3, figsize=(15, 6))
axes = axes.ravel()
ylabels = ['$M_B$', '$M_G$', '$M_V$', '$M_R$', '$M_I$']
bands = [M_b, M_g, M_v, M_r, M_i]
for ax, i in zip(axes, np.arange(len(bands))):
    out = ax.hist2d(np.log10(medianDistMatched/1000.), bands[i], bins=500, norm=LogNorm(), cmap=plt.get_cmap('gray'))
    if i in [3, 4]: ax.set_xlabel('Distance [kpc]', fontsize=15)
    ax.set_ylabel(ylabels[i], fontsize=20)
    ax.set_xlim(-2, 1)

In [ ]:
tree = kdtree.cKDTree(data=zip(M_b, M_g, M_v, M_r, M_i))

In [ ]:
treeColor = kdtree.cKDTree(data=zip(M_v, B_V, g_r, r_i))

In [ ]:
def chisq(data, value, error):
    return np.sqrt(np.sum(((data - value)/error)**2.))/len(data)

In [ ]:
import pdb
#some magsMatched[matchedInd, sourceInd] have 0.0 error :/

def distMetric(sourceInd, matchedInd, p=False):
    colorChiSq = 0.0
    colors = [B_V, g_r, r_i]
    color_errors = [['e_bmag', 'e_vmag'], ['e_gmag', 'e_rmag'], ['e_rmag', 'e_imag']]
    for color, error in zip(colors, color_errors):
        #print mag, error, sourceInd, matchedInd
        colorChiSq += (color[sourceInd] - color[matchedInd])**2./\
                      (magsMatched[error[0]][sourceInd]**2. + magsMatched[error[1]][sourceInd]**2. + \
                       magsMatched[error[0]][matchedInd]**2.+ magsMatched[error[1]][matchedInd]**2.)
        if p: print colorChiSq
    absMagChiSq = (M_v[sourceInd] - M_v[matchedInd])**2./\
                   (magsMatched['e_vmag'][sourceInd]**2. + magsMatched['e_vmag'][matchedInd]**2. +\
                  (errMatched[sourceInd]*1e-3 *5./np.log(10)*medianDistMatched[sourceInd])**2. +\
                  (errMatched[matchedInd]*1e-3*5./np.log(10)*medianDistMatched[matchedInd])**2.)
    totChiSq = np.sqrt(colorChiSq + absMagChiSq)
    if p:
        print 'the total chi2: ',totChiSq
        print 'the color chi2: ' ,colorChiSq
        print 'the absmag chi2:',absMagChiSq
    #if np.sum(np.isinf(totChiSq)): pdb.set_trace()
    return totChiSq

In [ ]:
#check ChiSq are what you think they are, correct length, why inf?
sources = []
matches = []
nstars = 6
chisqThreshold = 0.1

for star in np.arange(nstars): #np.arange(len(M_v)):
    pts = [M_v[star], B_V[star], g_r[star], r_i[star]]
    treeDistNum, treeIndNum = treeColor.query(pts, k=200) #nNeighbors)
    treeIndNum = treeIndNum[1:]
    chisq = distMetric(star, treeIndNum, p=False) 
    #print chisq
    treeIndNum = treeIndNum[chisq < chisqThreshold]
    sources.append(star)
    matches.append(treeIndNum)

In [ ]:
#print sources, matches

In [ ]:
#for s, m in zip(sources, matches): print np.sum(np.in1d(tgasRave['source_id'], tgasMatched[s]['source_id'])) + np.sum(np.in1d(tgasRave['source_id'], tgasMatched[m]['source_id']))

In [ ]:
#for s, m in zip(sources, matches): print np.where(np.in1d(tgasRave['source_id'], tgasMatched[s])), \
#    np.where(np.in1d(tgasRave['source_id'], tgasMatched[m]['source_id']))

In [ ]:
indices = []
for s, m in zip(sources, matches): 
    indices.append(np.concatenate((np.where(np.in1d(tgasRave['source_id'], tgasMatched[s]['source_id']))[0], \
                                   np.where(np.in1d(tgasRave['source_id'], tgasMatched[m]['source_id']))[0])))
#print indices

In [ ]:
fig, ax = plt.subplots(nstars, 3, figsize=(15, nstars*5))
axes = itertools.cycle(ax.ravel())

for j in np.arange(len(indices)):
    ax = axes.next()
    twins = matches[j]
    star = sources[j]
    raveTwins = indices[j]

    ax.set_title(tgasMatched['tycho2_id'][star])
    ax.errorbar(B_V[star], g_r[star], 
                xerr = np.sqrt(magsMatched['e_bmag'][star]**2. + magsMatched['e_vmag'][star]**2.), 
                yerr = np.sqrt(magsMatched['e_gmag'][star]**2. + magsMatched['e_rmag'][star]**2.), fmt='o', color='black',lw=2)
    ax.errorbar(B_V[twins], g_r[twins], 
                xerr = np.sqrt(magsMatched['e_bmag'][twins]**2. + magsMatched['e_vmag'][twins]**2.), 
                yerr = np.sqrt(magsMatched['e_gmag'][twins]**2. + magsMatched['e_rmag'][twins]**2.), fmt=None, color='blue')
    #ax.scatter(B_V[treeIndNum], g_r[treeIndNum], c=colors, s=200)
    ax.scatter(B_V[star], g_r[star], s=400, c='black')
    ax.set_xlabel('B - V')
    ax.set_ylabel('g - r')

    ax = axes.next()
    ax.set_title(tgasMatched['tycho2_id'][star])
    ax.errorbar(r_i[star], M_v[star], 
                xerr = np.sqrt(magsMatched['e_rmag'][star]**2. + magsMatched['e_imag'][star]**2.), 
                yerr = np.sqrt(magsMatched['e_vmag'][star]**2. + \
                               (errMatched[star]*1e-3 *5./np.log(10)*medianDistMatched[star])**2.), fmt='o', color='black')
    ax.errorbar(r_i[twins], M_v[twins], 
                xerr = np.sqrt(magsMatched['e_rmag'][twins]**2. + magsMatched['e_imag'][twins]**2.), 
                yerr = np.sqrt(magsMatched['e_vmag'][twins]**2. + (errMatched[twins]*1e-3 *5./np.log(10)*medianDistMatched[twins])**2.),
                              fmt=None, color='blue')
    #ax.scatter(B_V[treeIndNum], M_v[treeIndNum], c=colors, s=100, lw=None)
    ax.scatter(r_i[star], M_v[star], s=400, c='black')
    ax.set_xlabel('r - i')
    ax.set_ylabel('M_v')
    
    #treeDistDis, treeIndDis = treeColor.query(pts, k=nNeighbors, distance_upper_bound=errDistance)
    #if not np.array_equal(treeIndNum, treeIndDis): print treeIndNum, treeIndDis
    #import pdb; pdb.set_trace()    
    
    #if star<10:print chisq
    ax = axes.next()
    ax.set_title(tgasMatched['tycho2_id'][s])
    ax.errorbar(tgasRave['TEFF'][raveTwins]/1000., tgasRave['LOGG'][raveTwins], xerr=tgasRave['E_TEFF'][raveTwins]/1000., yerr=tgasRave['E_LOGG'][raveTwins],
               fmt='o', color='black')
    ax.set_ylabel('log g', fontsize=15)
    ax.set_xlabel('Teff [kK]', fontsize=15)
    ax.set_xlim(4.5, 7)
    ax.set_ylim(3, 5)
    ax.grid()
plt.tight_layout()

In [ ]:
import matplotlib as mpl
mpl.rcParams.update({'font.size': 25}) 
for j, i in enumerate(indices):
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.errorbar(tgasRave['TEFF'][i]/1000., tgasRave['LOGG'][i], xerr=tgasRave['E_TEFF'][i]/1000., yerr=tgasRave['E_LOGG'][i],
               fmt='o', color='black')
    ax.set_ylabel('log g')
    ax.set_xlabel('Teff [kK]')
    ax.set_xticks([5, 6, 7])
    ax.set_xlim(4.5, 7)
    ax.set_ylim(3, 5)
    ax.grid()
    plt.tight_layout()
    fig.savefig('stellarTwins.' + str(j) + '.png')

In [ ]:
for i in indices:
    for j in i:
        text = 'Tycho2 Id: ' + str(tgasRave[j]['tycho2_id']) + '\n' + \
       'pmra: ' + "{:.2f}".format(np.float(tgasRave[j]['pmra'])) + '\n' + \
       'pmdc: ' + "{:.2f}".format(np.float(tgasRave[j]['pmdec'])) + '\n' + \
       'vrad: ' + "{:.2f}".format(np.float(tgasRave[j]['HRV']))
        print text